<a href="https://colab.research.google.com/github/popescuaaa/playground/blob/master/Simple_GAN_Improvement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data

# Setup

In [8]:
config = {
  'learning_rate': 0.1,

  'g': {
      'dim_latent': 128,
      'dim_hidden': 128,
      'act': 'relu',
      'norm': 'batch'
  },

  'd': {
      'act': 'relu',
      'norm': 'none'
  }
}

## Helpers

In [9]:
batch_size = 32
transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),]
)
dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw




Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


# Generator

In [ ]:
class Generator(nn.Module):
  def __init__(self, in_features):
    super().__init__()
    self.params = in_features
    __module_list = [
        nn.Linear(self.params['input_size'], self.params['hidden_layer_1']),
        nn.ReLU(),
        nn.Linear(self.params['hidden_layer_1'], self.params['hidden_layer_2']),
        nn.ReLU(),
        nn.Linear(self.params['hidden_layer_2'], self.params['output_size']),
        nn.Tanh()
    ]

    self.__net = nn.Sequential(*__module_list)

  def forward(self, x):
     return self.__net(x)

# Discriminator

In [ ]:
class Discriminator(nn.Module):
  def __init__(self, in_features):
    super().__init__()
    self.params = in_features
    __module_list = [
        nn.Linear(self.params['input_size'], self.params['hidden_layer_1']),
        nn.ReLU(),
        nn.Linear(self.params['hidden_layer_1'], self.params['hidden_layer_2']),
        nn.ReLU(),
        nn.Linear(self.params['hidden_layer_2'], self.params['output_size']),
        nn.Sigmoid()
    ]

    self.__net = nn.Sequential(*__module_list)

  def forward(self, x):
     return self.__net(x)

# System training